### In this notebook, we're going to cover some of the most fundamental concepts of tensors using TensorFlow

Mospecifically, we're going to cover:
+ Introduction to tensors
* Getting information from tensors
+ Manipulating tensors
+ Tensors & NumPy
+ Using @tf.function (a way to speed up your regular Python functions)
+ Using GPUs with TensorFlow or (TPUs)
+ Exercises to try for yourself

## Introduction to Tensors

In [1]:
# Import TensorFlow
import tensorflow as tf
print(tf.__version__)

2.9.0


In [2]:
# Create tensors with tf.constant()
scalar = tf.constant(7)
scalar

<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [3]:
# Check the number of dimensions of a tensor (ndim stands for number of dimensions)
scalar.ndim

0

In [4]:
# Create a vector
vector = tf.constant([10, 10])
vector

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10, 10])>

In [5]:
# Check the dimension of our vector
vector.ndim

1

In [6]:
# Create a matrix (has more than 1 dimension)
matrix = tf.constant([[10, 2], [20,3]])
matrix

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  2],
       [20,  3]])>

In [7]:
# Check the dimension of our matrix
matrix.ndim

2

In [8]:
# Create another matrix
another_matrix = tf.constant([[10., 7.],
                             [3., 2.],
                             [8., 9.]], dtype=tf.float16)

another_matrix

<tf.Tensor: shape=(3, 2), dtype=float16, numpy=
array([[10.,  7.],
       [ 3.,  2.],
       [ 8.,  9.]], dtype=float16)>

In [9]:
another_matrix.ndim

2

In [10]:
# Let's create a tensor
tensor = tf.constant([[[1,2,3],
                      [4,5,6]],
                     [[7,8,9],
                     [10,11,12]],
                     [[13,14,15],
                     [16,17,18]]])

tensor

<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[ 1,  2,  3],
        [ 4,  5,  6]],

       [[ 7,  8,  9],
        [10, 11, 12]],

       [[13, 14, 15],
        [16, 17, 18]]])>

In [11]:
tensor.ndim

3

What we've created so far:

* Scalar: a single number
* Vector: a number with direction (e.g. wind speed and direction)
* Matrix: a 2-dimensional array of numbers
* Tensor: an n-dimensional array of numbers (when n can be any number, a 0-dimensinal tensor is a scalar, a 1-dimensional tensor is a vector)

### Creating tensors with `tf.Variable`

In [12]:
tf.Variable

tensorflow.python.ops.variables.Variable

In [14]:
# Create the same tensor with tf.Variable as above
changeable_tensor = tf.Variable([10, 7])
unchangeable_tensor = tf.constant([10, 7])
changeable_tensor, unchangeable_tensor

(<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10,  7])>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([10,  7])>)

In [15]:
# Let's try change one of the elements in our changeable tensor
changeable_tensor[0]

<tf.Tensor: shape=(), dtype=int32, numpy=10>

In [16]:
changeable_tensor[0] = 7
changeable_tensor

TypeError: 'ResourceVariable' object does not support item assignment

In [17]:
# How about we try .assign()
changeable_tensor[0].assign(7)
changeable_tensor

<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([7, 7])>

In [19]:
# Let's try change our unchangeable tensor
unchangeable_tensor[0] = 7
unchangeable_tensor

TypeError: 'tensorflow.python.framework.ops.EagerTensor' object does not support item assignment

In [20]:
unchangeable_tensor[0].assign(7)
unchangeable_tensor

AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'assign'

### Creating random tensors

Random tensors are tensors of same abitrary size which contain random numbers.

In [23]:
# Create two random (but the same) tensors
random_1 = tf.random.Generator.from_seed(42) # set seed for reproducibility
random_1 = random_1.normal(shape=(3,2))
random_1

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[-0.7565803 , -0.06854702],
       [ 0.07595026, -1.2573844 ],
       [-0.23193763, -1.8107855 ]], dtype=float32)>

In [24]:
random_2 = tf.random.Generator.from_seed(42)
random_2 = random_2.normal(shape=(3,2))
random_2

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[-0.7565803 , -0.06854702],
       [ 0.07595026, -1.2573844 ],
       [-0.23193763, -1.8107855 ]], dtype=float32)>

In [25]:
# Are they equal?
random_1, random_2, random_1 == random_2

(<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193763, -1.8107855 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193763, -1.8107855 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=bool, numpy=
 array([[ True,  True],
        [ True,  True],
        [ True,  True]])>)

### Shuffle the order of elements in a tensor

In [28]:
# Shuffle a tensor (valuable for when you want to shuffle your data so the inherent order doesn't effect learning)
not_shuffled = tf.constant([[10, 7],
                           [3, 4],
                           [2,5]])
not_shuffled.ndim

2

In [29]:
not_shuffled

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]])>

In [30]:
# Shuffle our non-shuffled tensor
tf.random.shuffle(not_shuffled)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 2,  5],
       [10,  7],
       [ 3,  4]])>

In [34]:
# Shuffle our non-shuffled tensor
tf.random.shuffle(not_shuffled, seed=42)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]])>

In [36]:
tf.random.set_seed(42)
tf.random.shuffle(not_shuffled, seed=42)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]])>

In [37]:
not_shuffled_2 = tf.constant([[[2,3,4],
                              [7,9,3],
                              [51,2,4]],
                             [[5,3,4],
                             [57,8,9],
                             [2,3,1]],
                             [[98,0,6],
                             [54,36,7],
                             [89,96,7]]])


In [38]:
not_shuffled_2.ndim

3

In [39]:
not_shuffled_2

<tf.Tensor: shape=(3, 3, 3), dtype=int32, numpy=
array([[[ 2,  3,  4],
        [ 7,  9,  3],
        [51,  2,  4]],

       [[ 5,  3,  4],
        [57,  8,  9],
        [ 2,  3,  1]],

       [[98,  0,  6],
        [54, 36,  7],
        [89, 96,  7]]])>

In [41]:
tf.random.shuffle(not_shuffled_2)

<tf.Tensor: shape=(3, 3, 3), dtype=int32, numpy=
array([[[ 5,  3,  4],
        [57,  8,  9],
        [ 2,  3,  1]],

       [[98,  0,  6],
        [54, 36,  7],
        [89, 96,  7]],

       [[ 2,  3,  4],
        [ 7,  9,  3],
        [51,  2,  4]]])>

In [43]:
nS2 = tf.random.shuffle(not_shuffled_2, seed = 13)

In [46]:
nS2

<tf.Tensor: shape=(3, 3, 3), dtype=int32, numpy=
array([[[ 2,  3,  4],
        [ 7,  9,  3],
        [51,  2,  4]],

       [[98,  0,  6],
        [54, 36,  7],
        [89, 96,  7]],

       [[ 5,  3,  4],
        [57,  8,  9],
        [ 2,  3,  1]]])>

In [47]:
nS2.ndim

3

In [51]:
type(not_shuffled_2)

tensorflow.python.framework.ops.EagerTensor